# Without PiCam

In [ ]:
import cv2
import time
import requests
from twilio.rest import Client

bot_token = '7113580453:AAGhbXLJHAqUnZ573ywKfozzncK2P6f7598'
chat_id = '-4020286607'
url = f"https://api.telegram.org/bot{bot_token}/sendPhoto"
data = {'chat_id': chat_id}

account_sid = 'AC30707f16aa788a474eb99972b43c5f15'
auth_token = '723a8133b727f0c8f27c626894b73c99'

# Initialize Twilio client
client = Client(account_sid, auth_token)

# Your Twilio phone number (from Twilio dashboard)
from_number = '+14159697262'

# Recipient's phone number
to_number = '+919550246132'  # Include country code

# Your SMS message
message_body = 'Someone is following, Check Telegram!'

# Load the pre-trained face detection model
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Open the default camera (usually the webcam)
cap = cv2.VideoCapture(0)

start_time = None
face_saved = False

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Flag to check if any face is detected above 0.5 unit
    face_detected_above_threshold = False

    # Draw rectangles around the detected faces
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        
        estimated_distance = 150 / w  

        # Display the estimated distance
        cv2.putText(frame, f'{estimated_distance:.2f} units', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Check if any face is detected above 0.5 unit
        if estimated_distance > 0.5:
            face_detected_above_threshold = True

    # If any face is detected above 0.5 unit, start the timer
    if face_detected_above_threshold:
        if start_time is None:
            start_time = time.time()
        elif time.time() - start_time > 3:  # Save the image if the time is above 3 seconds
            if not face_saved:
                filename = f'face_at_distance_{estimated_distance:.2f}_units.jpg'
                cv2.imwrite(filename, frame)
                print(f"Saved image: {filename}")
                face_saved = True
                
                #sending alert message to mobile
                message = client.messages.create(body=message_body,from_=from_number,to=to_number)
                
                #sending images to telegram
                files = {'photo': open(filename, 'rb')}
                response = requests.post(url, files=files, data=data)
                
    else:
        start_time = None
        face_saved = False

    # Display the frame with the detected faces
    cv2.imshow('Face Detection', frame)

    # Check for the 'q' key to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture
cap.release()
cv2.destroyAllWindows()

# With PiCam

In [ ]:
import cv2
import time
import os 
import numpy as np
import requests
from twilio.rest import Client
from picamera2 import Picamera2

cam = Picamera2()
face_detector=cv2.CascadeClassifier('/home/raspberry/Desktop/FaceRecognition new/haarcascade_frontalface_default.xml')
cam.preview_configuration.main.size = (640, 360)
cam.preview_configuration.main.format = "RGB888"
cam.preview_configuration.controls.FrameRate=30
cam.preview_configuration.align()
cam.configure("preview")
cam.start()

start_time = None
face_saved = False

bot_token = '6544811186:AAGS_YLl3ZFaw1dWgBkaSV4l_ktaE-uvjIo'
chat_id = '-4020286607'
url = f"https://api.telegram.org/bot{bot_token}/sendPhoto"
data = {'chat_id': chat_id}

account_sid = 'AC30707f16aa788a474eb99972b43c5f15'
auth_token = '723a8133b727f0c8f27c626894b73c99'

# Initialize Twilio client
client = Client(account_sid, auth_token)

# Your Twilio phone number (from Twilio dashboard)
from_number = '+14159697262'

# Recipient's phone number
to_number = '+919550246132'  # Include country code

# Your SMS message
message_body = 'Someone is following, Check Telegram!'

while True:
    frame=cam.capture_array()
    frameGray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_detector.detectMultiScale(frameGray,scaleFactor=1.1,minNeighbors=5,minSize=(150, 150))

    # Flag to check if any face is detected above 0.5 unit
    face_detected_above_threshold = False

    # Draw rectangles around the detected faces
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        
        estimated_distance = 150 / w
        
        # Display the estimated distance
        cv2.putText(frame, f'{estimated_distance:.2f} units', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Check if any face is detected above 0.5 unit
        if estimated_distance > 0.5:
            face_detected_above_threshold = True

    # If any face is detected above 0.5 unit, start the timer
    if face_detected_above_threshold:
        if start_time is None:
            start_time = time.time()
        elif time.time() - start_time > 3:  # Save the image if the time is above 3 seconds
            if not face_saved:
                filename = f'face_at_distance_{estimated_distance:.2f}_units.jpg'
                cv2.imwrite(filename, frame)
                print(f"Saved image: {filename}")
                face_saved = True
                
                #sending alert message to mobile
                message = client.messages.create(body=message_body,from_=from_number,to=to_number)
                
                #sending images to telegram
                files = {'photo': open(filename, 'rb')}
                response = requests.post(url, files=files, data=data)
                
    else:
        start_time = None
        face_saved = False

    cv2.imshow('Face Detection',frame)
    key = cv2.waitKey(100) & 0xff
    
    if key == 27:
        break
    elif key == 113:
        break

print("\n [INFO] Exiting Program and cleaning up stuff")
cam.stop()
cv2.destroyAllWindows()